In [ ]:
!nvidia-smi

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Unzip

In [ ]:
!unzip /content/drive/MyDrive/Trash_obj/data_original.zip -d /content  # images

In [ ]:
!unzip /content/drive/MyDrive/Trash_obj/masks.zip -d /content  # masks

In [ ]:
!unzip /content/drive/MyDrive/Trash_obj/new_image.zip -d /content 

In [ ]:
!unzip /content/drive/MyDrive/Trash_obj/new_mask.zip -d /content 

In [ ]:
!unzip /content/drive/MyDrive/Trash_obj/battery_plus.json.zip -d /content 

# Install Requierd Packages

In [ ]:
import torch
torch.__version__

'1.8.1+cu101'

In [ ]:
!pip install bottleneck-transformer-pytorch
!pip install timm
!pip install albumentations==0.4.6
!pip install adamp
!pip install mmcv-full -f https://download.openmmlab.com/mmcv/dist/cu101/torch1.8.1/index.html
!pip install mmdet

In [ ]:
!pip uninstall pycocotools -y
!pip install mmpycocotools

In [ ]:
%cd /content/drive/MyDrive/Trash_obj/apex-master

In [ ]:
!pip install -v --disable-pip-version-check --no-cache-dir ./

# cd

In [ ]:
%cd /content/drive/MyDrive/Trash_obj/UniverseNet-master/

# BaseLine Code

### Import

In [ ]:
from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)

In [ ]:
!nvidia-smi --gpu-reset

### Seed, Batch, Epoch, GPU set

In [ ]:
cfg.data.samples_per_gpu = 2 # Batch Size
cfg.runner.max_epochs = 45 # Epochs
cfg.seed=42 # Seed
cfg.gpu_ids = [0] # GPU set

### Class, Dataset set

In [ ]:
# Classes Set
classes = ("UNKNOWN", "General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")
# config file 들고오기
cfg = Config.fromfile('./configs/detectors/detectors_htc_r50_1x_coco.py')
PREFIX = '/content/'
cfg.data_root = PREFIX

# dataset 바꾸기
cfg.data.train.classes = classes
cfg.data.train.img_prefix = PREFIX
cfg.data.train.ann_file = PREFIX + 'battery_plus.json'
# cfg.data.train.pipeline[2]['img_scale'] = (512, 512)

cfg.data.val.classes = classes
cfg.data.val.img_prefix = PREFIX
cfg.data.val.ann_file = PREFIX + 'val.json'
cfg.data.val.pipeline[1]['img_scale'] = (512, 512)

cfg.data.test.classes = classes
cfg.data.test.img_prefix = PREFIX
cfg.data.test.ann_file = PREFIX + 'test.json'
cfg.data.test.pipeline[1]['flip'] = True
cfg.data.test.pipeline[1]['img_scale'] = [(256, 256),(320, 320),(384, 384),(448, 448),
                      (512, 512),(576, 576),(640, 640),(704, 704),(768, 768)]

# Mask, BBox classes 설정
cfg.model.roi_head.mask_head[0]['num_classes'] = 11
cfg.model.roi_head.mask_head[1]['num_classes'] = 11
cfg.model.roi_head.mask_head[2]['num_classes'] = 11

cfg.model.roi_head.bbox_head[0]['num_classes'] = 11
cfg.model.roi_head.bbox_head[1]['num_classes'] = 11
cfg.model.roi_head.bbox_head[2]['num_classes'] = 11

### Model, Neck BackBone Set

In [ ]:
# Model BackBone
cfg.model.pretrained='open-mmlab://resnext101_32x4d' # 32 or 64
cfg.model.backbone=dict(
        type='DetectoRS_ResNeXt',
        # pretrained='open-mmlab://resnext101_32x4d', # 32 or 64
        depth=101,
        groups=32, # 32 or 64
        base_width=4,
        num_stages=4,
        out_indices=(0,1,2,3),
        frozen_stages=1,
        norm_cfg=dict(type='BN', requires_grad=True),
        norm_eval=True,
        conv_cfg=dict(type='ConvAWS'),
        sac=dict(type='SAC', use_deform=True),
        stage_with_sac=(False, True, True, True),
        output_img=True,
        style='pytorch')

# Neck BackBone
cfg.model.neck=dict(
        type='RFP',
        rfp_steps=2,
        in_channels=[256, 512, 1024, 2048],
        out_channels=256,
        num_outs=5,
        aspp_out_channels=64,
        aspp_dilations=(1, 3, 6, 1),
        rfp_backbone=dict(
            rfp_inplanes=256,
            type='DetectoRS_ResNeXt',
            depth=101,
            groups=32, # 32 or 64
            base_width=4,
            num_stages=4,
            out_indices=(0, 1, 2, 3),
            frozen_stages=1,
            norm_cfg=dict(type='BN', requires_grad=True),
            norm_eval=True,
            conv_cfg=dict(type='ConvAWS'),
            sac=dict(type='SAC', use_deform=True),
            stage_with_sac=(False, True, True, True),
            pretrained='open-mmlab://resnext101_32x4d', # 32 or 64
            style='pytorch'))

### Augmentation

In [ ]:
# Albumentation Augmentations
cfg.albu_train_transforms = [
    dict(
        type='VerticalFlip',
        p=0.5),
    dict(
        type='RandomRotate90'),
]

# Train Pipeline
cfg.train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations', with_bbox=True, with_mask=True, with_seg=True),
    dict(type='RandomFlip', flip_ratio=0.5),
    # Multi Scale, Random Crop Training
    dict(type='AutoAugment',
         policies=[
             [
                 dict(type='Resize',
                      img_scale=[(512, 512),(576, 576),(640, 640),(704, 704),(768, 768)],
                      multiscale_mode='value',
                      keep_ratio=True)
             ],
             [
                 dict(type='Resize',
                      img_scale=[(512, 512), (768, 768)],
                      multiscale_mode='value',
                      keep_ratio=True),
                 dict(type='RandomCrop',
                      crop_type='absolute_range',
                      crop_size=(384, 384),
                      allow_negative_crop=True),
                 dict(type='Resize',
                      img_scale=[(512, 512),(576, 576),(640, 640),(704, 704),(768, 768)],
                      multiscale_mode='value',
                      override=True,
                      keep_ratio=True)
             ]
         ]),
    dict(type='Pad', size_divisor=32),
    # Albumentation Augmentations, Get config from 'albu_train_transforms'
    dict(
        type='Albu',
        transforms=albu_train_transforms,
        bbox_params=dict(
            type='BboxParams',
            format='pascal_voc',
            label_fields=['gt_labels'],
            min_visibility=0.0,
            filter_lost_elements=True),
        keymap={
            'img': 'image',
            'gt_masks': 'masks',
            'gt_bboxes': 'bboxes'
        },
        update_pad_shape=False,
        skip_img_without_anno=True),
    dict(type='Normalize', **img_norm_cfg),
    dict(type='SegRescale', scale_factor=1 / 8),
    dict(type='DefaultFormatBundle'),
    dict(type='Collect', keys=['img', 'gt_bboxes', 'gt_labels', 'gt_masks', 'gt_semantic_seg']),
]

### LR, Scheduler, Optimizer set

In [ ]:
# Scheduler
cfg.lr_config = dict(
  policy='CosineAnnealing',
  min_lr=1e-10,
  warmup='linear',
  warmup_iters=5,
  warmup_ratio=1e-10,
  warmup_by_epoch=True
)

# Optimizer, lr
cfg.optimizer = dict(type='AdamW', lr=0.0001, weight_decay=0.01)
cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)

# Hook
cfg.fp16 = None
cfg.optimizer_config = dict(
    type='ApexOptimizerHook',
    # Batch size x Update Interval = Normalize Term
    update_interval=4,
    grad_clip=None,
    coalesce=True,
    bucket_size_mb=-1,
    use_fp16=True,
)

### Resume Epoch

In [ ]:
cfg.resume_from = '/content/drive/MyDrive/Trash_obj/UniverseNet-master/work_dirs/detectors/DetectoRS_mstrain_400_1200_x101_64x4d_40e/epoch_28.pth'

# Initialize Model, Dataset

In [ ]:
model = build_detector(cfg.model)

In [ ]:
datasets = [build_dataset(cfg.data.train)]

# Train

In [ ]:
train_detector(model, datasets[0], cfg, distributed=False, validate=True)

# Inference

In [ ]:
%cd /content/drive/MyDrive/Trash_obj/UniverseNet-master/

/content/drive/MyDrive/Trash_obj/UniverseNet-master


### Import

In [ ]:
import mmcv
from mmcv import Config
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.models import build_detector
from mmdet.apis import single_gpu_test
from mmcv.runner import load_checkpoint
import os
from mmcv.parallel import MMDataParallel
import pandas as pd
from pandas import DataFrame
from pycocotools.coco import COCO
import numpy as np

### Seed, Batch, Epoch, GPU set

In [ ]:
cfg.data.samples_per_gpu = 4 
epoch = 45 # Epoch which Weight to Inference
cfg.seed=42
cfg.gpu_ids = [0]

In [ ]:
classes = ("UNKNOWN", "General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")
# config file 들고오기
cfg = Config.fromfile('./configs/detectors/detectors_htc_r50_1x_coco.py')

PREFIX = '/content/'

# dataset 바꾸기
cfg.data.train.classes = classes
cfg.data.train.img_prefix = PREFIX
cfg.data.train.ann_file = PREFIX + 'train.json'
# cfg.data.train.pipeline[2]['img_scale'] = (512, 512)

cfg.data.val.classes = classes
cfg.data.val.img_prefix = PREFIX
cfg.data.val.ann_file = PREFIX + 'val.json'
cfg.data.val.pipeline[1]['img_scale'] = (512, 512)

cfg.data.test.classes = classes
cfg.data.test.img_prefix = PREFIX
cfg.data.test.ann_file = PREFIX + 'test.json'
cfg.data.test.pipeline[1]['flip'] = True
# TTA Multi Scales
cfg.data.test.pipeline[1]['img_scale'] = [(512, 512),(576, 576),(640, 640),(704, 704),(768, 768)]

# Set Model same as trained
cfg.model.pretrained='open-mmlab://resnext101_32x4d'
cfg.model.backbone=dict(
        type='DetectoRS_ResNeXt',
        # pretrained='open-mmlab://resnext101_32x4d',
        depth=101,
        groups=32,
        base_width=4,
        num_stages=4,
        out_indices=(0,1,2,3),
        frozen_stages=1,
        norm_cfg=dict(type='BN', requires_grad=True),
        norm_eval=True,
        conv_cfg=dict(type='ConvAWS'),
        sac=dict(type='SAC', use_deform=True),
        stage_with_sac=(False, True, True, True),
        output_img=True,
        style='pytorch')

cfg.model.neck=dict(
        type='RFP',
        rfp_steps=2,
        in_channels=[256, 512, 1024, 2048],
        out_channels=256,
        num_outs=5,
        aspp_out_channels=64,
        aspp_dilations=(1, 3, 6, 1),
        rfp_backbone=dict(
            rfp_inplanes=256,
            type='DetectoRS_ResNeXt',
            depth=101,
            groups=32,
            base_width=4,
            num_stages=4,
            out_indices=(0, 1, 2, 3),
            frozen_stages=1,
            norm_cfg=dict(type='BN', requires_grad=True),
            norm_eval=True,
            conv_cfg=dict(type='ConvAWS'),
            sac=dict(type='SAC', use_deform=True),
            stage_with_sac=(False, True, True, True),
            pretrained='open-mmlab://resnext101_32x4d',
            style='pytorch'))

# Saved Directory
cfg.work_dir = './work_dirs/detectors/DetectoRS_mstrain_400_1200_x101_64x4d_40e'

# Same as Trained
cfg.model.roi_head.mask_head[0]['num_classes'] = 11
cfg.model.roi_head.mask_head[1]['num_classes'] = 11
cfg.model.roi_head.mask_head[2]['num_classes'] = 11

cfg.model.roi_head.bbox_head[0]['num_classes'] = 11
cfg.model.roi_head.bbox_head[1]['num_classes'] = 11
cfg.model.roi_head.bbox_head[2]['num_classes'] = 11

cfg.optimizer = dict(type='AdamW', lr=0.0001, weight_decay=0.01)
cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.model.train_cfg = None

# checkpoint path
checkpoint_path = os.path.join(cfg.work_dir, f'epoch_{epoch}.pth')

### Set Dataset, Model

In [ ]:
dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
        dataset,
        samples_per_gpu=1,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)

In [ ]:
model = build_detector(cfg.model, test_cfg=cfg.get('test_cfg'))
checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu')

model.CLASSES = dataset.CLASSES
model = MMDataParallel(model.cuda(), device_ids=[0])

### Outputs

In [ ]:
output = single_gpu_test(model, data_loader, show_score_thr=0.05)

### Submission File

In [ ]:
prediction_strings = []
file_names = []
coco = COCO(cfg.data.test.ann_file)
imag_ids = coco.getImgIds()

class_num = 11
for i, out in enumerate(output):
    prediction_string = ''
    image_info = coco.loadImgs(coco.getImgIds(imgIds=i))[0]
    for j in range(class_num):
        # 1 stage model = for o in out[j]:
        # 2 stage model Segmentation = for o in out[1][j]:
        # 2 stage model Detection
        for o in out[0][j]:
            prediction_string += str(j) + ' ' + str(o[4]) + ' ' + str(o[0]) + ' ' + str(o[1]) + ' ' + str(
                o[2]) + ' ' + str(o[3]) + ' '
        
    prediction_strings.append(prediction_string)
    file_names.append(image_info['file_name'])


submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names

# Submission File Name set
submission.to_csv(os.path.join(cfg.work_dir, f'submission_DetectoRS_ResNeXt101_Final_{epoch}.csv'), index=None)
submission.head()

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


,PredictionString,image_id
0,0 0.0012175218 268.1394 0.09808382 324.08347 3...,batch_01_vt/0021.jpg
1,0 0.005388239 286.18134 340.91245 308.2169 376...,batch_01_vt/0028.jpg
2,1 0.99937457 71.53811 170.43152 373.08047 377....,batch_01_vt/0031.jpg
3,0 0.0011801156 0.038686067 226.98409 34.433064...,batch_01_vt/0032.jpg
4,0 0.03611503 38.37911 460.02594 76.998695 472....,batch_01_vt/0070.jpg
